In [1]:
#VGG16

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoded format
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Example of using VGG16 for CIFAR-10 classification
vgg_model = tf.keras.applications.VGG16(weights=None, input_shape=(32, 32, 3), classes=10)

# Compile and train the model (using CIFAR-10 data)
#vgg_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


def train_model(model, X_train, y_train, X_test, y_test, model_name='VGG16', dataset_name="CIFAR10", total_epoch=50, ea=False, eamonitor="val_loss"):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')
    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor=eamonitor, patience=10, min_delta=0.001, verbose=1)

    start_time = time.time()
    if ea:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback,early_stopping])
    else:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='VGG16', dataset_name="CIFAR10", threshold=0.5):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test, threshold)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels, threshold=0.5):
    interpredictions = predictions.argmax(axis=1)  # Convert one-hot encoded predictions to class labels
    true_labels = true_labels.argmax(axis=1)  # Convert one-hot encoded true labels to class labels

    correct_predictions = np.sum(np.abs(true_labels - interpredictions) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, interpredictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, interpredictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels, interpredictions, average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }

# Train the model
model, history, predictions, training_time = train_model(vgg_model, x_train, y_train, x_test, y_test, model_name='VGG16', dataset_name='CIFAR10', total_epoch=10, ea=True)

# Test the model
test_metrics = test_model(model, x_test, y_test, model_name='VGG16', dataset_name='CIFAR10')

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 408s 259ms/step - accuracy: 0.1002 - loss: 2.3036 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 408s 261ms/step - accuracy: 0.0968 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 453s 290ms/step - accuracy: 0.1005 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3027
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 421s 269ms/step - accuracy: 0.1011 - loss: 2.3028 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 419s 268ms/step - accuracy: 0.1014 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 417s 267ms/step - accuracy: 0.1021 - loss: 2.3026 - val_accuracy: 0.1000 - val_loss: 2.3027
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 422s 270ms/step - accuracy: 0.0992 - loss: 2.3028 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 422s 270ms/step - ac